In [614]:
from tensorflow import keras
from sklearn.preprocessing import StandardScaler
from datetime import datetime
import os
import pandas as pd
import numpy as np

In [615]:
untouched_csv = pd.read_csv('C:/Users/Krupam/Downloads/data.csv', sep=',')

In [616]:
#replace the direction column with the corresponding degree values
direction_map = {'N': 0,'NNE': 22.5,'NE': 45,'ENE': 67.5,'E': 90,'ESE': 112.5,'SE': 135,'SSE': 157.5,
            'S': 180,'SSW': 202.5,'SW': 225,'WSW': 247.5,'W': 270,'WNW': 292.5,'NW': 315,'NNW': 337.5
        }

In [617]:
untouched_csv['Direction (A)'] = untouched_csv['Direction (A)'].replace(direction_map)

In [618]:
#convert to Celsius
untouched_csv['Mainland Air Temp'] = round((untouched_csv['Mainland Air Temp']-32) * 5.0/9.0, 1)
untouched_csv['LBI Air Temp'] = round((untouched_csv['LBI Air Temp']-32) * 5.0/9.0, 1)


In [619]:
#rounds all colomns
untouched_csv['Humidity (%)'] = round(untouched_csv['Humidity (%)'], 1)
untouched_csv['Wind Speed (A)'] = round(untouched_csv['Wind Speed (A)'], 1)
untouched_csv['Gusting'] = round(untouched_csv['Gusting'], 1)
untouched_csv['Atmospheric Pressure (IN)'] = round(untouched_csv['Atmospheric Pressure (IN)'], 2)
untouched_csv['Precipitation Rate'] = round(untouched_csv['Precipitation Rate'], 2)
untouched_csv['Bay Temp'] = round(untouched_csv['Bay Temp'], 2)
untouched_csv['Salinity'] = round(untouched_csv['Salinity'], 2)
untouched_csv['LBI Air Temp'] = round(untouched_csv['LBI Air Temp'], 1)
untouched_csv['Ocean Temp'] = round(untouched_csv['Ocean Temp'], 1)

In [620]:
#determines if its a onshore breeze and adds a new column for it
onshore_degrees = [180, 135, 157.5, 90, 67.5, 45, 22.5]
untouched_csv['Onshore'] = untouched_csv['Direction (A)'].isin(onshore_degrees)
untouched_csv["Onshore"] = untouched_csv["Onshore"].astype(int)



In [621]:
#thresholds
ocean_thresh = 1.0
bay_thresh = 1.67
wind_thresh = 7.0
untouched_csv['rolling avg'] = untouched_csv['Bay Temp'].rolling(window=24).mean()
#change the 1 to change the temperature threshold
untouched_csv['bay_bool'] = (abs(untouched_csv['Bay Temp'] - untouched_csv['rolling avg']) > bay_thresh).astype(int)
untouched_csv['ocean_bool'] = (abs(untouched_csv["Bay Temp"] - untouched_csv["Ocean Temp"]) > ocean_thresh).astype(int)
untouched_csv['big_wind'] = (untouched_csv['Wind Speed (A)'] > wind_thresh).astype(int)
untouched_csv["upwelling_flag"] = ((untouched_csv['bay_bool'] == 1) & (untouched_csv['ocean_bool'] == 1) & (untouched_csv['big_wind'] == 1)).astype(int)
	
	
    

In [622]:
#untouched_csv.head()

num_upwelling_events = untouched_csv['upwelling_flag'].sum()
print("Number of upwelling hours:", num_upwelling_events)
upwelling_rows = untouched_csv[untouched_csv['upwelling_flag'] == 1]
print(upwelling_rows)


#print(untouched_csv.to_string())


Number of upwelling hours: 20
      Date  Hr  Mainland Air Temp  Humidity (%) Direction (A)  Wind Speed (A)  \
255   6/11  15               29.7          43.0         202.5             9.4   
280   6/12  16               35.4          35.0           225             7.3   
281   6/12  17               34.7          36.0         247.5             7.3   
449   6/19  17               33.7          57.0         202.5            11.3   
450   6/19  18               33.2          58.0           180             7.7   
592   6/25  16               38.6          43.0           180            10.4   
593   6/25  17               36.9          48.0           180             7.8   
610   6/26  10               28.1          76.0            45             7.4   
619   6/26  19               19.7          80.0          67.5             9.4   
620   6/26  20               19.2          82.0          67.5             8.1   
621   6/26  21               18.9          84.0            45             8.6  